## Select stock symbol and date range

In [ ]:
symbol = 'NETE'
start_date = "2020-12-11"
end_date = "2020-12-12"
anchored_vwaps_start = ['2020-12-11 09:30:00', '2020-12-11 14:20:00']

#### Imports

In [ ]:
# silence warnings
import warnings
import numpy as np
import pandas as pd
import pytz
import plotly.graph_objects as go
import chart_studio.plotly as py
from datetime import datetime
import alpaca_trade_api as tradeapi
from liualgotrader.common.market_data import get_symbol_data
from liualgotrader.fincalcs.vwap import anchored_vwap
from liualgotrader.fincalcs.resample import resample, ResampleRangeType
%matplotlib inline
warnings.filterwarnings("ignore")
est = pytz.timezone("US/Eastern")

#### Load symbol data

In [ ]:
start_date = est.localize(datetime.strptime(start_date, "%Y-%m-%d"))
end_date = est.localize(datetime.strptime(end_date, "%Y-%m-%d"))

In [ ]:
ohlc_data = get_symbol_data(symbol, start_date, end_date)
ohlc_data

In [ ]:
if ohlc_data is None or ohlc_data.empty:
    assert False, "No data loaded"

## Visuals

In [ ]:
anchored_vwap_starts = [
    datetime.strptime(anchored_vwap_start, "%Y-%m-%d %H:%M:%S").replace(tzinfo=est)
    for anchored_vwap_start in anchored_vwaps_start
]
org_ohlc_data = ohlc_data

for sample_rate in (
    ResampleRangeType.min_1,
    ResampleRangeType.min_5,
    ResampleRangeType.min_15,
):
    ohlc_data = resample(org_ohlc_data, sample_rate)
    anchored_vwap_indicators = [
        anchored_vwap(ohlc_data, anchored_vwap_start)
        for anchored_vwap_start in anchored_vwap_starts
    ]

    trace1 = {
        "x": ohlc_data.index,
        "open": ohlc_data.open,
        "high": ohlc_data.high,
        "low": ohlc_data.low,
        "close": ohlc_data.close,
        "type": "candlestick",
        "name": f"{symbol} {sample_rate} bars",
        "yaxis": "y2",
        "showlegend": True,
    }
    trace2 = [
        {
            "x": anchored_vwap_indicator.index,
            "y": anchored_vwap_indicator,
            "type": "scatter",
            "mode": "lines",
            "line": {"width": 2, "color": "black"},
            "yaxis": "y2",
            "name": f"VWAP-{indx+1}",
            "showlegend": True,
        }
        for indx, anchored_vwap_indicator in enumerate(anchored_vwap_indicators)
    ]
    fig = dict(data=[trace1], layout=dict())

    fig["layout"]["plot_bgcolor"] = "rgb(200, 200, 200)"
    fig["layout"]["xaxis"] = dict(rangeselector=dict(visible=True))
    fig["layout"]["yaxis"] = dict(domain=[0, 0.2], showticklabels=False)
    fig["layout"]["yaxis2"] = dict(domain=[0.2, 0.8])
    fig["layout"]["legend"] = dict(
        orientation="h",
        y=0.95,
        x=0.3,
        yanchor="bottom",
    )
    fig["layout"]["margin"] = dict(t=40, b=40, r=40, l=40)

    rangeselector = dict(
        # visibe = True,
        x=0,
        y=0.9,
        bgcolor="rgba(150, 200, 250, 0.4)",
        font=dict(size=13),
        buttons=list(
            [
                dict(count=1, label="1 yr", step="year"),
                dict(count=3, label="3 mo", step="month", stepmode="backward"),
                dict(count=1, label="1 mo", step="month", stepmode="backward"),
                dict(count=7, label="1 wk", step="day", stepmode="backward"),
                dict(step="all"),
            ]
        ),
    )

    fig["layout"]["xaxis"]["rangeselector"] = rangeselector

    fig["data"] += trace2

    colors = []

    for i in range(len(ohlc_data.close)):
        if i != 0:
            if ohlc_data.close[i] > ohlc_data.close[i - 1]:
                colors.append("green")
            else:
                colors.append("red")
        else:
            colors.append("red")

    fig["data"].append(
        dict(
            x=ohlc_data.index,
            y=ohlc_data.volume,
            marker=dict(color=colors),
            type="bar",
            yaxis="y",
            name="Volume",
            showlegend=False,
        )
    )

    f = go.Figure(data=fig["data"], layout=fig["layout"])
    f.show()